# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


In [50]:
!pip install --quiet langchain

In [51]:
!pip install --quiet langchain_experimental langchain_openai langchain_community

In [52]:
!pip install faiss-cpu > /dev/null
!pip install google-search-results > /dev/null

## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [53]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [54]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [55]:
import os
#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv())

from google.colab import userdata
OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY_Ironhack')
SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')

In [64]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [57]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)

todo_chain = LLMChain(llm=OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), prompt=todo_prompt)
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [20]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use create_react_agent instead.
  warn_deprecated(


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [21]:
OBJECTIVE = "Write a weather report for SF today"

In [22]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [23]:
baby_agi({"objective": OBJECTIVE})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should create a todo list for the objective
Action: TODO
Action Input: Write a weather report for SF today
Observation: 

1. Check the current weather conditions in San Francisco.
2. Gather information on the temperature, humidity, wind speed, and precipitation.
3. Research any potential weather patterns or systems that may affect the current conditions.
4. Look at historical data and trends for this time of year in San Francisco.
5. Consult with local meteorologists or weather experts for their insights and predictions.
6. Create a rough outline or structure for the weather report, including sections for current conditions, forecast, and any notable weather events.
7. Begin writing the report, starting with a brief overview of the current conditions.
8. Use descriptive language and visuals (such as maps or charts) to enhance the report.
9. Include any 

{'objective': 'Write a weather report for SF today'}

In [24]:

OBJECTIVE = "Generate a weather forecast for the next 2 days"
baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

2: Check for any weather alerts or warnings in the area.
3: Research current weather conditions in San Francisco.
4: Create a list of key points to include in the weather report.
5: Write a brief introduction for the weather report.
6: Include a section on the current temperature and expected high and low for the day.
7: Provide information on the current humidity levels.
8: Mention any potential changes in weather throughout the day.
9: Include a section on wind speed and direction.
10: Write a conclusion summarizing the overall weather conditions for San Francisco today.
11: Share the weather report with the appropriate audience.
12: Check for any updates on the weather conditions in SF.
13: Monitor the temperature throughout the day and update the report accordingly.
14: Research and include any additional information on heat-related illnesses and precautions.
15: Include a section on the UV index and recommended sun protection.
16: Provide a detailed breakdown

{'objective': 'Generate a weather forecast for the next 2 days'}

# My changes

I changed the embeddings to BertModel

In [30]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [59]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)

summarize_prompt = PromptTemplate.from_template("You are an expert in summarizing texts to a certain length. Summarize the following text: {text}")
translate_prompt = PromptTemplate.from_template("You are an expert in translating texts to a certain language. Translate the following text: {text}")

todo_chain = LLMChain(llm=OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), prompt=todo_prompt)
summarize_chain = LLMChain(llm=OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), prompt=summarize_prompt)
translate_chain = LLMChain(llm=OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), prompt=translate_prompt)


search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
    Tool(
        name="Summarization",
        func=summarize_chain.run,
        description="Summarizes text or documents into concise summaries."
    ),
    Tool(
        name="Translation",
        func=translate_chain.run,
        description="Translates text from one language to another."
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [60]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [65]:
# Logging of LLMChains
verbose = True
# If None, will keep on going forever
max_iterations: Optional[int] = 5
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [66]:

OBJECTIVE = "Generate a plan for 2 days in Amsterdam for a group of 4 people, summarize the text into 400 characters and translate it into Portuguese."
baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to create a plan for 2 days in Amsterdam for a group of 4 people
Action: TODO
Action Input: Create a plan for 2 days in Amsterdam for a group of 4 people
Observation: 

1. Research and book flights to Amsterdam for the group of 4 people.
2. Reserve accommodations for 2 nights in Amsterdam, taking into consideration the group's budget and preferred location.
3. Create a budget for the trip, including transportation, accommodations, food, and activities.
4. Plan out a rough itinerary for the 2 days, taking into account the group's interests and must-see attractions in Amsterdam.
5. Purchase tickets in advance for any popular attractions or activities to avoid long lines.
6. Research and make reservations for any desired restaurants or dining experiences.
7. Familiarize yourself with the public transportation system in Amsterdam and plan out the most 

{'objective': 'Generate a plan for 2 days in Amsterdam for a group of 4 people, summarize the text into 400 characters and translate it into Portuguese.'}